# Train model to detect images with an obstacle on track

Author : Johan Jublanc
    
Date : 20/09/2019

Description : 
- Two type of data are collected : the first ones without any obstacle on the track, the other one with an obstacle
- A pretrained model is downloaded and several layer are unfrozen so than the top convolutional layers can be retrained
- The model make a prediction that can be used on a portion of the image (image cropped

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
print("TensorFlow version is ", tf.__version__)

from tensorflow.keras.metrics import Precision
from tensorflow.keras.metrics import Accuracy

import mlflow
import mlflow.tensorflow
import mlflow.keras

import numpy as np

import random

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.model_selection import train_test_split

from xebikart.images import transformer as T

import xebikart.dataset as dataset

In [ ]:
tf.enable_eager_execution()

## Params

In [ ]:
n_epochs = 20
batch_size = 32
learning_rate = 0.001
decay = 0.00005

model_name = "detection_model"

## Get the data

In [ ]:
# parameters
# dataset parameters
tubes_root_folder = "file:/workspace/xebikart-ml-tubes"
tubes_folders = [
    "tub.v4.02",
    "tub.v6.01"
]
test_size=0.2

In [ ]:
raw_tubes_df = dataset.get_tubes_df(tubes_root_folder, tubes_folders, tubes_extension=".tar.gz")
tubes_df = raw_tubes_df.rename(columns={"cam/image_array": "images_path", "user/angle": "angles", "user/throttle": "throttles"})
tubes_df.count()

In [ ]:
images_path = tubes_df["images_path"].tolist()
labels = tubes_df["num_tube"].tolist()

### The paths are split into a train group and a test group

In [ ]:
train_images_path, test_images_path, train_labels, test_labels = train_test_split(images_path, labels, test_size=test_size)
print('Train set :', len(train_images_path), 'images')
print('Test set :', len(test_images_path), 'images')

## Use the API data setto unzip and preprocess the data

The model take (160,160) normalized images. The following function process the data to fit the input of the model

In [ ]:
def resize_normalize(image_path):
    tf_image = T.read_image(image_path)
    tf_image = T.normalize(tf_image)
    tf_image = tf.image.resize(tf_image,(160,160), method = 2)
    return tf_image

Then the API dataset is used to process and shuffle the data and make batches.

In [ ]:
batch_size = 32

def input_fn(filepath, label, BATCH_SIZE = 32, SHUFFLE_SIZE = 200, NUM_EPOCHS = 50):
    ds_x = tf.data.Dataset.from_tensor_slices(filepath)
    ds_x = ds_x.map(resize_normalize)
    ds_y = tf.data.Dataset.from_tensor_slices(label)
    ds_x_y = tf.data.Dataset.zip((ds_x, ds_y)).shuffle(SHUFFLE_SIZE).repeat(NUM_EPOCHS).batch(BATCH_SIZE).prefetch(1)
    
    return ds_x_y

In [ ]:
ds_train = input_fn(train_images_path, train_labels, BATCH_SIZE = batch_size, SHUFFLE_SIZE = 100, NUM_EPOCHS = n_epochs)
ds_test = input_fn(test_images_path, test_labels)

### Take a look to the images

In [ ]:
images_path = tubes_df["images_path"]
labels = tubes_df["num_tube"]

In [ ]:
random_image_path = tubes_df.sample()["images_path"].values[0]


tf_image_original = T.read_image(random_image_path)
tf_image_normalized = T.normalize(tf_image_original)
tf_image_resized = tf.image.resize(tf_image_normalized,(160,160), method = 2)

fig, axs = plt.subplots(1, 3, figsize=(15,15), constrained_layout=True)
axs[0].set_title("Original")
axs[0].imshow(tf_image_original)
axs[1].set_title("normalized")
axs[1].imshow(tf_image_normalized)
axs[2].set_title("resized")
axs[2].imshow(tf_image_resized)
plt.show()

# Improve a pretrained model to classify our images

### Use a pretrained model

##### Import the model

In [ ]:
zip_file_url = "https://storage.googleapis.com/download.tensorflow.org/models/tflite/mobilenet_v1_1.0_224_quant_and_labels.zip"

In [ ]:
import requests, zipfile, io
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [ ]:
image_size = 160
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

##### Choose the number of layer to train

In [ ]:
nb_layer_train = 5
nb_layer_freez = len(base_model.layers) - nb_layer_train

In [ ]:
for layer in base_model.layers[:nb_layer_freez]:
    layer.trainable = False

##### Let's take a look at the base model architecture

In [ ]:
base_model.summary()

##### Add an output dense layer

In [ ]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

##### Choose the parameters and compile

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr = learning_rate, 
                                                 decay = decay),
              loss='binary_crossentropy',
              metrics=[Precision(), Accuracy()])

In [ ]:
model.summary()

In [ ]:
len(model.trainable_variables)

## Train and evaluate the model

### Train

In [ ]:
mlflow.set_experiment("convolutional_neural_network")

with mlflow.start_run():
    mlflow.log_params({
        "images": str(tubes_folders),
        "nb_images": len(train_images_path),
        "epochs": n_epochs,
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "decay": decay
    })
    mlflow.tensorflow.autolog()
    history = model.fit(x = ds_train,
                    steps_per_epoch = len(train_labels)//batch_size,
                    epochs = n_epochs,
                    verbose = 1,
                    validation_data = ds_test,
                    validation_steps = len(test_labels)//batch_size)

#### Evaluate

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save(model_name + '.h5')

#### Vizualize the results

In [ ]:
model = load_model(model_name + '.h5')

In [ ]:
def get_tf_image_resized(im_path):
    tf_image_original = T.read_image(im_path)
    tf_image_normalized = T.normalize(tf_image_original)
    tf_image_resized = tf.image.resize(tf_image_normalized,(160,160), method = 2)
    tf_img = tf.reshape(tf_image_resized,(1,160,160,3))
    
    return tf_img

In [ ]:
random_image_path_0 = tubes_df[tubes_df.num_tube==0].sample()["images_path"].values[0]
random_image_path_1 = tubes_df[tubes_df.num_tube==1].sample()["images_path"].values[0]

img_0 = get_tf_image_resized(random_image_path_0)
img_1 = get_tf_image_resized(random_image_path_1)
    

fig, axs = plt.subplots(1, 2, figsize=(15,15), constrained_layout=True)
axs[0].set_title("Clear (0) : Predict ({})".format(model.predict(img_0)))
axs[0].imshow(T.read_image(random_image_path_0))
axs[1].set_title("Obstacle (1) : Predict ({})".format(model.predict(img_1)))
axs[1].imshow(T.read_image(random_image_path_1))
plt.show()

## Try to detect an obstacle in a box

In [ ]:
def get_tf_image_resized_cropped(im_path,box):
    tf_image_original = T.read_image(im_path)
    tf_image_cropped = tf_image_original[box[0]:box[2], box[1]:box[3]]
    tf_image_normalized = T.normalize(tf_image_cropped)
    tf_image_resized = tf.image.resize(tf_image_normalized,(160,160), method = 2)
    tf_img = tf.reshape(tf_image_resized,(1,160,160,3))
    
    return tf_img

#### Show an image and a blue on the same picture

In [ ]:
top = 60
left = 4
win_heigh = 58
win_width = 150
box = (top, left, top + win_heigh, left + win_width)

In [ ]:
random_image_path_0 = tubes_df[tubes_df.num_tube==0].sample()["images_path"].values[0]
random_image_path_1 = tubes_df[tubes_df.num_tube==1].sample()["images_path"].values[0]

img_crop0 = get_tf_image_resized_cropped(random_image_path_0,box)
img_crop1 = get_tf_image_resized_cropped(random_image_path_1,box)
    

fig, axs = plt.subplots(1, 2, figsize=(15,15), constrained_layout=True)
axs[0].set_title("Clear (0) : Predict ({})".format(model.predict(img_crop0)))
axs[0].imshow(T.read_image(random_image_path_0))
axs[0].plot([box[1],box[1]],[box[0],box[2]], color="C0",linewidth=4)
axs[0].plot([box[3],box[3]],[box[0],box[2]], color="C0",linewidth=4)
axs[0].plot([box[1],box[3]],[box[0],box[0]], color="C0",linewidth=4)
axs[0].plot([box[1],box[3]],[box[2],box[2]], color="C0",linewidth=4)

axs[1].set_title("Obstacle (1) : Predict ({})".format(model.predict(img_crop1)))
axs[1].imshow(T.read_image(random_image_path_1))
axs[1].plot([box[1],box[1]],[box[0],box[2]], color="C0",linewidth=4)
axs[1].plot([box[3],box[3]],[box[0],box[2]], color="C0",linewidth=4)
axs[1].plot([box[1],box[3]],[box[0],box[0]], color="C0",linewidth=4)
axs[1].plot([box[1],box[3]],[box[2],box[2]], color="C0",linewidth=4)
plt.show()